In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline
np.random.seed(1)

In [2]:
import os
os.getcwd()

'/home/dileep/Desktop/ripo'

# Importing and Cleaning table

In [3]:
tnn = pd.read_csv('/home/dileep/Desktop/ripo/NN_source_file.csv', index_col=[0])
tnn.shape

(1647, 34)

In [4]:
tnn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1647 entries, 1 to 1647
Data columns (total 34 columns):
Name                  1647 non-null object
CPMRN                 1629 non-null object
Month.of.Admission    1647 non-null object
Age                   1647 non-null int64
Gender                1647 non-null object
Hospital              1647 non-null object
History               1647 non-null object
Surgery               1647 non-null object
Vent.mode             1623 non-null object
GCS                   1647 non-null int64
Temparature           1647 non-null object
HR                    1647 non-null float64
SpO2                  1647 non-null int64
SBP                   1647 non-null float64
MAP                   1647 non-null float64
RR                    1647 non-null int64
FiO2                  1647 non-null float64
PaO2                  1647 non-null float64
PaCO2                 1647 non-null float64
pH                    1647 non-null float64
A.a.gradient          1647 non

In [5]:
tnn['Temparature'] = pd.to_numeric(tnn['Temparature'], errors='coerce')
tnn.Temparature = tnn.Temparature.replace(np.nan, 98.6)
sum(tnn.Temparature.isnull())

0

In [6]:
tnn['Month.of.Admission'].unique()

array(['July 2018', 'August 2018', 'September 2018', 'October 2018',
       'November 2018', 'December 2018', 'January 2019', 'February 2019',
       'July 0201', 'January 2018', 'August 2015', 'February 2018',
       'February 0219'], dtype=object)

In [7]:
tnn['Month.of.Admission'] = tnn['Month.of.Admission'].replace('July 0201', 'July 2018')
tnn['Month.of.Admission'] = tnn['Month.of.Admission'].replace('January 2018', 'January 2019')
tnn['Month.of.Admission'] = tnn['Month.of.Admission'].replace('August 2015', 'August 2018')
tnn['Month.of.Admission'] = tnn['Month.of.Admission'].replace('February 2018', 'February 2019')
tnn['Month.of.Admission'] = tnn['Month.of.Admission'].replace('February 0219', 'February 2019')

In [8]:
tnn['Vent.mode'].value_counts()

RA           555
Mask         407
Cannula      332
ETT          171
High Flow     95
Trach         63
Name: Vent.mode, dtype: int64

In [9]:
tnn['Vent.mode'] = tnn['Vent.mode'].replace(np.nan, 'RA')

In [10]:
tnn2 = tnn.drop(['Name', 'CPMRN', 'History', 'SBP'], axis=1)

In [11]:
tnn_trim = tnn.drop(['Name', 'CPMRN', 'History', 'SBP', 'Month.of.Admission', 'Gender', 'Hospital'], axis=1)

In [12]:
tnn2.columns

Index(['Month.of.Admission', 'Age', 'Gender', 'Hospital', 'Surgery',
       'Vent.mode', 'GCS', 'Temparature', 'HR', 'SpO2', 'MAP', 'RR', 'FiO2',
       'PaO2', 'PaCO2', 'pH', 'A.a.gradient', 'HCO3', 'Hb', 'TLC', 'Platelets',
       'K', 'Na', 'Serum.Cr', 'Blood.Urea', 'Bili', 'Urine.output', 'Lactate',
       'INR', 'Survival'],
      dtype='object')

In [13]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
def scaleColumns(df, cols_to_scale):
    for col in cols_to_scale:
        df[col] = pd.DataFrame(scaler.fit_transform(pd.DataFrame(df[col])))
    return df
tnn_scaled = scaleColumns(tnn2, ['Age', 'GCS', 'Temparature', 'HR', 'SpO2', 'MAP', 'RR', 'FiO2', 
                                          'PaO2', 'PaCO2', 'pH', 'A.a.gradient', 'HCO3', 'Hb', 'TLC', 'Platelets', 
                                         'K', 'Na', 'Serum.Cr', 'Blood.Urea', 'Bili', 'Urine.output', 'Lactate', 
                                         'INR'])
tnt_scaled = scaleColumns(tnn_trim, ['Age', 'GCS', 'Temparature', 'HR', 'SpO2', 'MAP', 'RR', 'FiO2', 
                                          'PaO2', 'PaCO2', 'pH', 'A.a.gradient', 'HCO3', 'Hb', 'TLC', 'Platelets', 
                                         'K', 'Na', 'Serum.Cr', 'Blood.Urea', 'Bili', 'Urine.output', 'Lactate', 
                                         'INR'])

/home/dileep/anaconda3/envs/cpcv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/dileep/anaconda3/envs/cpcv/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/dileep/anaconda3/envs/cpcv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/dileep/anaconda3/envs/cpcv/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/dileep/anaconda3/envs/cpcv/lib/python3.7/site-packages/sklearn/preprocessing/d

### Changing dtypes of numerics from float64 to float32

In [14]:
names = np.array(tnn_scaled.columns)
for i in range(len(names)):
    if tnn_scaled[names[i]].dtype == 'float64':
        tnn_scaled[names[i]] = tnn_scaled[names[i]].astype('float32')
names2 = np.array(tnt_scaled.columns)
for i in range(len(names2)):
    if tnt_scaled[names2[i]].dtype == 'float64':
        tnt_scaled[names2[i]] = tnt_scaled[names2[i]].astype('float32')
tnn_scaled.info()
tnt_scaled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1647 entries, 1 to 1647
Data columns (total 30 columns):
Month.of.Admission    1647 non-null object
Age                   1646 non-null float32
Gender                1647 non-null object
Hospital              1647 non-null object
Surgery               1647 non-null object
Vent.mode             1647 non-null object
GCS                   1646 non-null float32
Temparature           1646 non-null float32
HR                    1646 non-null float32
SpO2                  1646 non-null float32
MAP                   1646 non-null float32
RR                    1646 non-null float32
FiO2                  1646 non-null float32
PaO2                  1646 non-null float32
PaCO2                 1646 non-null float32
pH                    1646 non-null float32
A.a.gradient          1646 non-null float32
HCO3                  1646 non-null float32
Hb                    1646 non-null float32
TLC                   1646 non-null float32
Platelets         

In [15]:
tnn3 = pd.get_dummies(tnn_scaled, drop_first=True)
tnt3 = pd.get_dummies(tnt_scaled, drop_first=True)

In [16]:
tnn3 = tnn3.dropna()
tnt3 = tnt3.dropna()

In [17]:
print(tnn3.columns)
print(tnt3.columns)

Index(['Age', 'GCS', 'Temparature', 'HR', 'SpO2', 'MAP', 'RR', 'FiO2', 'PaO2',
       'PaCO2', 'pH', 'A.a.gradient', 'HCO3', 'Hb', 'TLC', 'Platelets', 'K',
       'Na', 'Serum.Cr', 'Blood.Urea', 'Bili', 'Urine.output', 'Lactate',
       'INR', 'Month.of.Admission_December 2018',
       'Month.of.Admission_February 2019', 'Month.of.Admission_January 2019',
       'Month.of.Admission_July 2018', 'Month.of.Admission_November 2018',
       'Month.of.Admission_October 2018', 'Month.of.Admission_September 2018',
       'Gender_Male', 'Hospital_Brains-MVJ', 'Hospital_Brains-SSNMC',
       'Hospital_HCG DR', 'Hospital_HCG EKO', 'Hospital_HCG Hubli',
       'Hospital_HCG Vijayawada', 'Hospital_MRNH', 'Hospital_Prashant',
       'Surgery_Emergency surgery', 'Surgery_Non-Operative', 'Vent.mode_ETT',
       'Vent.mode_High Flow', 'Vent.mode_Mask', 'Vent.mode_RA',
       'Vent.mode_Trach', 'Survival_Expired'],
      dtype='object')
Index(['Age', 'GCS', 'Temparature', 'HR', 'SpO2', 'MAP', 'RR', 'FiO

In [18]:
print(tnn3.shape)
print(tnt3.shape)

(1646, 48)
(1646, 32)


In [19]:
X = tnn3.iloc[:, 0:47].values; Y = tnn3.iloc[:, 47].values.reshape(-1, 1); 
print(X.shape); print(Y.shape);

(1646, 47)
(1646, 1)


In [20]:
x = tnt3.iloc[:, 0:31].values; y = tnt3.iloc[:, 31].values.reshape(-1, 1);
print(x.shape); print(y.shape);

(1646, 31)
(1646, 1)


In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, stratify=Y)

In [22]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, stratify=y)

## Keras Model
## Performance in full data

In [23]:
from keras.layers import Dense, Dropout
from keras.models import Sequential
import matplotlib.pyplot as plt
from keras import regularizers
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

Using TensorFlow backend.


In [24]:
model = Sequential()
model.add(Dense(300, activation='relu', input_shape=(47,)))
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(300, kernel_regularizer=regularizers.l2(0.01), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(300, kernel_regularizer=regularizers.l2(0.01), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(300, kernel_regularizer=regularizers.l2(0.01), activation= 'relu'))
model.add(Dropout(0.5))
model.add(Dense(50, kernel_regularizer=regularizers.l2(0.1), activation='relu'))
model.add(Dense(2, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [25]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [26]:
model.fit(X_train, Y_train, batch_size=100, epochs=20)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
823/823 [==============================] - 2s 2ms/step - loss: 16.6490 - acc: 0.9429
Epoch 2/20
823/823 [==============================] - 0s 158us/step - loss: 13.4323 - acc: 0.9526
Epoch 3/20
823/823 [==============================] - 0s 159us/step - loss: 10.7043 - acc: 0.9526
Epoch 4/20
823/823 [==============================] - 0s 199us/step - loss: 8.4385 - acc: 0.9526
Epoch 5/20
823/823 [==============================] - 0s 172us/step - loss: 6.6018 - acc: 0.9526
Epoch 6/20
823/823 [==============================] - 0s 156us/step - loss: 5.1644 - acc: 0.9526
Epoch 7/20
823/823 [==============================] - 0s 162us/step - loss: 4.0045 - acc: 0.9526
Epoch 8/20
823/823 [==============================] - 0s 183us/step - loss: 3.1150 - acc: 0.9526
Epoch 9/20
823/823 [==============================] - 0s 270us/step - loss: 2.4149 - acc: 0.9526
Epoch 10/20
823/823 [==============================] - 0s 219us/step - loss: 1

In [27]:
Y_tr_pred_prob = model.predict(X_train)
Y_ts_pred_prob = model.predict(X_test)

In [28]:
y_tr_pred_prob = Y_tr_pred_prob[:,1]
y_ts_pred_prob = Y_ts_pred_prob[:,1]
print(roc_auc_score(Y_train, y_tr_pred_prob))
print(roc_auc_score(Y_test, y_ts_pred_prob))

0.9991496598639457
0.6261813537675607


## Performance in trim data

In [29]:
model = Sequential()
model.add(Dense(300, activation='relu', input_shape=(31,)))
model.add(Dropout(0.5))
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(300, activation='relu'))
model.add(Dense(300, activation='relu'))
model.add(Dense(300, kernel_regularizer=regularizers.l2(0.01), activation='relu'))
model.add(Dense(300, kernel_regularizer=regularizers.l2(0.01), activation= 'relu'))
model.add(Dense(50, kernel_regularizer=regularizers.l2(0.1), activation='relu'))
model.add(Dense(2, activation='softmax'))

In [30]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [31]:
model.fit(x_train, y_train, batch_size=100, epochs=20)

Epoch 1/20
823/823 [==============================] - 1s 1ms/step - loss: 13.9340 - acc: 0.9332
Epoch 2/20
823/823 [==============================] - 0s 151us/step - loss: 11.1702 - acc: 0.9514
Epoch 3/20
823/823 [==============================] - 0s 150us/step - loss: 8.7894 - acc: 0.9514
Epoch 4/20
823/823 [==============================] - 0s 147us/step - loss: 6.8594 - acc: 0.9514
Epoch 5/20
823/823 [==============================] - 0s 143us/step - loss: 5.3093 - acc: 0.9514
Epoch 6/20
823/823 [==============================] - 0s 150us/step - loss: 4.0956 - acc: 0.9514
Epoch 7/20
823/823 [==============================] - 0s 144us/step - loss: 3.1367 - acc: 0.9514
Epoch 8/20
823/823 [==============================] - 0s 151us/step - loss: 2.4148 - acc: 0.9514
Epoch 9/20
823/823 [==============================] - 0s 152us/step - loss: 1.8555 - acc: 0.9514
Epoch 10/20
823/823 [==============================] - 0s 146us/step - loss: 1.4243 - acc: 0.9514
Epoch 11/20
823/823 [========

In [32]:
Y_tr_pp = model.predict(x_train)
Y_ts_pp = model.predict(x_test)
y_tr_pred_prob = Y_tr_pp[:,1]
y_ts_pred_prob = Y_ts_pp[:,1]
print(roc_auc_score(y_train, y_tr_pred_prob))
print(roc_auc_score(y_test, y_ts_pred_prob))

0.9597381864623243
0.58899136577708


## Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [ ]:
c_space = np.logspace(-5, 8, 15)
param_grid = {'C': c_space}

In [ ]:
logreg = LogisticRegression()
logreg_cv = GridSearchCV(logreg, param_grid, cv=5)

In [ ]:
logreg_cv.fit(x_train, y_train)

In [ ]:
print("Tuned LR parameters: {}".format(logreg_cv.best_params_))
print("Best score is: {}".format(logreg_cv.best_score_))